In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM
from tensorflow.keras.models import Model

In [6]:
class Encoder(Model):
    def __init__(self,vocab_size,embed_dim,enc_units):
        super().__init__()
        self.embedding=Embedding(vocab_size,embed_dim)
        self.lstm=LSTM(enc_units,return_state=True)
    
    def call(self,x):
        x=self.embedding(x)
        _,c,h=self.lstm(x)
        return c,h
        

In [7]:
class Decoder(Model):
    def __init__(self,vocab_size,embed_dim,dec_units):
        super().__init__()
        self.embedding=Embedding(vocab_size,embed_dim)
        self.lstm=LSTM(dec_units,return_state=True,return_sequences=True)
        self.fc=tf.keras.layers.Dense(vocab_size)
    
    def call(self,x,hidden,cell):
        x=self.embedding(x)
        outputs,c,h=self.lstm(x,initial_state=[hidden,cell])
        outputs=self.fc(outputs)
        return outputs,h,c


In [21]:
# Encoder input (batch_size = 1)
input_seq = tf.constant([[1, 2]])      # "I eat"

# Decoder input (teacher forcing)
target_seq = tf.constant([[0, 1, 2]])  # "<SOS> I eat"
inp_vocab_size = 10
tar_vocab_size = 10


In [22]:

encoder = Encoder(inp_vocab_size, 256, 512)
decoder = Decoder(tar_vocab_size, 256, 512)

enc_h, enc_c = encoder(input_seq)
dec_outputs, _, _ = decoder(target_seq, enc_h, enc_c)


In [23]:
dec_outputs


<tf.Tensor: shape=(1, 3, 10), dtype=float32, numpy=
array([[[ 2.7041598e-03,  3.7028051e-03, -2.8534620e-03,  6.9285696e-03,
         -1.3146694e-03,  6.4831423e-03,  4.7664354e-03,  3.0706641e-03,
          2.7089329e-03,  3.6540064e-03],
        [ 4.1895974e-03,  7.5073009e-03,  1.5428150e-04,  6.4722975e-03,
         -4.3027997e-03,  8.9374734e-03,  1.3768589e-03,  1.0859391e-03,
          3.3311895e-03,  5.0618644e-03],
        [ 3.2579293e-04,  2.3239593e-03, -3.0662036e-03,  5.5756746e-04,
         -1.6524456e-05,  1.3383193e-02,  1.4699677e-03,  3.1274348e-03,
         -2.1492639e-03, -1.1019888e-02]]], dtype=float32)>

ValueError: in user code:

    File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_10372\3129603076.py", line 9, in train_step  *
        loss = loss_fn(dec_out, predictions)
    File "c:\Users\Administrator\Desktop\RNN\venv\Lib\site-packages\keras\src\losses\loss.py", line 67, in __call__  **
        losses = self.call(y_true, y_pred)
    File "c:\Users\Administrator\Desktop\RNN\venv\Lib\site-packages\keras\src\losses\losses.py", line 28, in call
        y_true_y_pred = tree.map_structure(
    File "c:\Users\Administrator\Desktop\RNN\venv\Lib\site-packages\keras\src\tree\tree_api.py", line 200, in map_structure
        return tree_impl.map_structure(func, *structures, none_is_leaf=none_is_leaf)
    File "c:\Users\Administrator\Desktop\RNN\venv\Lib\site-packages\keras\src\tree\optree_impl.py", line 111, in map_structure
        return optree.tree_map(
    File "c:\Users\Administrator\Desktop\RNN\venv\Lib\site-packages\optree\ops.py", line 766, in tree_map
        return treespec.

AttributeError: 'tuple' object has no attribute 'shape'